пересборка датасета только для теста из-за ошибки в query для теста

In [1]:
import pandas as pd
import os
from tqdm import tqdm_notebook

In [2]:
save_folder = '../../data/combined_dataset/'
!mkdir -p $save_folder

In [3]:
#train_normal_filename = '../../data/prepaired_dataset/train_v3.fth'
test_normal_filename = '../../data/prepaired_dataset/test_v3.fth'

In [4]:
quantiles_folder = '../../data/coms_sep/quantiles_data'
quantiles_train_files = [os.path.join(quantiles_folder, x) for x in os.listdir(quantiles_folder) 
                         if (x.count('qnts_train')) and (not x.count('amount.csv'))]
quantiles_test_files = [os.path.join(quantiles_folder, x) for x in os.listdir(quantiles_folder)
                        if (x.count('qnts_test')) and (not x.count('amount.csv'))]
assert len(quantiles_train_files) == len(quantiles_test_files)

---

загружаем обычный датасет с подсчитанными признаками

In [5]:
#train_normal = pd.read_feather(train_normal_filename)
test_normal = pd.read_feather(test_normal_filename)

In [6]:
#train_normal['event_time'] = train_normal['event_time'].astype(str)
test_normal['event_time'] = test_normal['event_time'].astype(str)

---

загружаем woe

In [7]:
#train_woe = pd.read_csv('../../data/coms_sep/train_woe.csv', index_col = 0, low_memory = False)
test_woe = pd.read_csv('../../data/coms_sep/test_woe.csv', index_col = 0, low_memory = False)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


---

обьединяем все датасеты с квантилями в один датафрейм

In [8]:
def unite_quantiles(files):
    quantiles_list = [pd.read_csv(x, low_memory = False, index_col = 0) for x in files]
    uniq_ids_dates = lambda q_list: pd.concat([x[['user_id', 'short_date']] for x in q_list]).drop_duplicates().sort_values(['user_id', 'short_date']).reset_index().drop('index', axis = 1)
    ids_dates = uniq_ids_dates(quantiles_list)
    ids_dates_merged = ids_dates.copy()
    for quant_df in tqdm_notebook(quantiles_list):
        cols_for_na_fill = [x for x in quant_df.columns if not (x in ['user_id', 'short_date'])]
        ids_dates_merged = ids_dates_merged.merge(quant_df, how = 'left', on = ['user_id', 'short_date'])
        ids_dates_merged.loc[:, cols_for_na_fill] = ids_dates_merged.loc[:, cols_for_na_fill].fillna(-1000)
    return ids_dates_merged

In [9]:
%%time
#train_quantiles = unite_quantiles(quantiles_train_files)
test_quantiles = unite_quantiles(quantiles_test_files)

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))


CPU times: user 6.96 s, sys: 3.17 s, total: 10.1 s
Wall time: 10.3 s


In [10]:
%%time
#train_quantiles.drop_duplicates(['user_id', 'short_date'], inplace = True)
test_quantiles.drop_duplicates(['user_id', 'short_date'], inplace = True)

CPU times: user 180 ms, sys: 209 ms, total: 389 ms
Wall time: 387 ms


----

соединяем normal с woe

In [11]:
test_woe = test_woe.drop(['event_description'], axis = 1)
test_woe.rename(columns = {x: f'woe_{x}' for x in  test_woe.columns if not x in ['event_time', 'user_id']},
                 inplace = True)

In [12]:
#train_woe.drop_duplicates(['user_id', 'event_time'], inplace = True)
test_woe.drop_duplicates(['user_id', 'event_time'], inplace = True)

In [13]:
%%time
te_with_woe = test_normal.merge(test_woe, how = 'left', on = ['user_id', 'event_time'])
te_woe_cols = [x for x in te_with_woe.columns if x.startswith('woe_')]
te_with_woe.loc[:, te_woe_cols] = te_with_woe.loc[:, te_woe_cols].fillna(0.0)

CPU times: user 4.83 s, sys: 3.02 s, total: 7.86 s
Wall time: 7.85 s


In [14]:
#assert tr_with_woe.shape[0] == train_normal.shape[0]
assert te_with_woe.shape[0] == test_normal.shape[0]

----

добавляем квантили

In [15]:
%%time
te_combined = te_with_woe.merge(test_quantiles, how = 'left', on = ['user_id', 'short_date'])
te_combined_quant_cols = [x for x in te_combined.columns if x.count('_days_quantile_')]
te_combined.loc[:, te_combined_quant_cols] = te_combined.loc[:, te_combined_quant_cols].fillna(-1000)

CPU times: user 3.04 s, sys: 4.34 s, total: 7.38 s
Wall time: 7.38 s


In [16]:
#assert tr_combined.shape[0] == train_normal.shape[0]
assert te_combined.shape[0] == test_normal.shape[0]

----

сохраняем все

In [17]:
%%time
#tr_combined.to_feather(save_folder + 'train_v1.fth')
te_combined.to_feather(save_folder + 'test_v1.fth')

CPU times: user 821 ms, sys: 1.07 s, total: 1.89 s
Wall time: 1.34 s
